In [1]:
######################################################################################################################################################
# Importación de librerías, modelos y rutas                                                                                                          #
######################################################################################################################################################

import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.mixed_precision import set_global_policy

# Configuración de precisión mixta
set_global_policy('mixed_float16')

######################################################################################################################################################
# Configuración de rutas y carga de datos                                                                                                           #
######################################################################################################################################################

# Ruta de imágenes
image_path = "./data-set/"
# Ruta de los CSV
information_path = "./information/"

2025-01-15 13:20:01.118995: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736968801.266028    2922 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736968801.307707    2922 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-15 13:20:01.639663: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Cargar el CSV
dataset = pd.read_csv("dataset-vectors.csv")

# Convertir la columna `image_vector` de texto a lista de floats
dataset['image_vector'] = dataset['image_vector'].apply(lambda x: np.array(eval(x), dtype=np.float32))

# Codificar las clases
label_encoder = LabelEncoder()
dataset['galaxy_class_encoded'] = label_encoder.fit_transform(dataset['galaxy_class'])
num_classes = len(label_encoder.classes_)

# Preparar los datos para entrenamiento
X = np.stack(dataset['image_vector'].values)
y = to_categorical(dataset['galaxy_class_encoded'], num_classes)

# Dividir en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
######################################################################################################################################################
# Preprocesamiento de datos para la RNN                                                                                                             #
######################################################################################################################################################

# Redimensionar y normalizar los datos
# Para una RNN, necesitamos que cada vector sea tratado como una secuencia
X_train_rnn = X_train.astype('float16') / 255.0
X_val_rnn = X_val.astype('float16') / 255.0

# Ajustar las dimensiones para que sean compatibles con la RNN (timesteps, features)
X_train_rnn = X_train_rnn.reshape(-1, 64, 64)
X_val_rnn = X_val_rnn.reshape(-1, 64, 64)

######################################################################################################################################################
# Creación y configuración de la RNN                                                                                                                #
######################################################################################################################################################

# Crear el modelo RNN
model = Sequential([
    LSTM(128, return_sequences=True, activation='relu', input_shape=(64, 64)),
    LSTM(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax', dtype='float32')
])
# Compilar el modelo con optimizador de escalado de pérdidas
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

######################################################################################################################################################
# Entrenamiento del modelo                                                                                                                          #
######################################################################################################################################################

history = model.fit(
    X_train_rnn, y_train,
    epochs=10,
    validation_data=(X_val_rnn, y_val),
    batch_size=8
)

W0000 00:00:1736968917.371980    2922 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/josafat/Documentos/galaxy-classifier/galaxy-classifier-venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1571/1571 ━━━━━━━━━━━━━━━━━━━━ 104s 65ms/step - accuracy: 0.3151 - loss: 1.5482 - val_accuracy: 0.3069 - val_loss: 1.4995
Epoch 2/10
1571/1571 ━━━━━━━━━━━━━━━━━━━━ 100s 64ms/step - accuracy: 0.3192 - loss: 1.5030 - val_accuracy: 0.3276 - val_loss: 1.5001
Epoch 3/10
1571/1571 ━━━━━━━━━━━━━━━━━━━━ 99s 63ms/step - accuracy: 0.3278 - loss: 1.5025 - val_accuracy: 0.3069 - val_loss: 1.5012
Epoch 4/10
1571/1571 ━━━━━━━━━━━━━━━━━━━━ 98s 62ms/step - accuracy: 0.3162 - loss: 1.5083 - val_accuracy: 0.3276 - val_loss: 1.5006
Epoch 5/10
1571/1571 ━━━━━━━━━━━━━━━━━━━━ 101s 64ms/step - accuracy: 0.3163 - loss: 1.4979 - val_accuracy: 0.3069 - val_loss: 1.5007
Epoch 6/10
1571/1571 ━━━━━━━━━━━━━━━━━━━━ 100s 64ms/step - accuracy: 0.3226 - loss: 1.4990 - val_accuracy: 0.3276 - val_loss: 1.4999
Epoch 7/10
1571/1571 ━━━━━━━━━━━━━━━━━━━━ 101s 64ms/step - accuracy: 0.3106 - loss: 1.5008 - val_accuracy: 0.3069 - val_loss: 1.4999
Epoch 8/10
1571/1571 ━━━━━━━━━━━━━━━━━━━━ 100s 64ms/step - accuracy: 0.

In [6]:
######################################################################################################################################################
# Guardar el modelo para futuras predicciones                                                                                                       #
######################################################################################################################################################

# Guardar el modelo entrenado
model.save("galaxy_classifier_rnn.h5")

# Guardar las etiquetas codificadas
with open("label_encoder_classes.npy", "wb") as f:
    np.save(f, label_encoder.classes_)

In [10]:
# Cargar el modelo entrenado
model = load_model("galaxy_classifier_rnn.h5")

# Cargar las etiquetas codificadas
with open("label_encoder_classes.npy", "rb") as f:
    classes = np.load(f, allow_pickle=True)

import cv2

# Cargar y preprocesar la nueva imagen
image_path = "./data-set/191718.jpg"
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (64, 64))
image = image.astype('float16') / 255.0
image = image.reshape(1, 64, 64)

# Realizar la predicción
predictions = model.predict(image)

# Mostrar todas las probabilidades para cada clase
print("Probabilidades para cada clase:")
for i, class_prob in enumerate(predictions[0]):
    print(f"{classes[i]}: {class_prob:.4f}")

# Obtener la clase con la mayor probabilidad
predicted_class_index = np.argmax(predictions)
predicted_class = classes[predicted_class_index]

print(f"\nClase predicha: {predicted_class} (índice: {predicted_class_index})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
Probabilidades para cada clase:
Ec: 0.3683
Er: 0.3253
SBb2m: 0.1990
SBc3m: 0.0356
Sb+t: 0.0174
Sc+t: 0.0543

Clase predicha: Ec (índice: 0)


In [14]:
# Cargar el CSV
dataset = pd.read_csv("dataset-vectors.csv")
imagen=dataset[dataset["image_name"]==191718]
imagen["image_vector"].iloc[0]
print(imagen)

       image_name                                       image_vector  \
15003      191718  [0.011764705882352941, 0.011764705882352941, 0...   

      galaxy_class  
15003           Ec  
